<a target="_blank" href="https://colab.research.google.com/github/andrew-nash/CS6421-labs/blob/main/CS6421_Assignment2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!pip install wandb -qU
!wget https://github.com/gmprovan/CS6421-Assignment1/raw/master/train.zip
!wget https://github.com/gmprovan/CS6421-Assignment1/raw/master/test.zip
!wget https://github.com/gmprovan/CS6421-Assignment1/raw/master/train_cleaned.zip
!unzip train.zip
!unzip test.zip
!unzip train_cleaned.zip

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tboard

# Setup

In [ ]:
cs={
    "task1": {
        "_descrip" : "",
        "permitted_layers" : ["InputLayer","Dense","BatchNormalization","Reshape","Flatten", "Dropout", "Conv2D", "MaxPooling2D", "MaxPooling1D", "UpSampling2D"],
        "optimizers" : ["SGD", "Adam", "Nadam","Adadelta","RMSprop"],
        "act_funcs" : ["relu","leaky_relu","sigmoid","elu","linear"],
        "min_lr" :  0.000001,
        "max_lr" :  0.1,
        "layer_limits" : {
            "Dense" : 30
        },
        "layer_mins" : {
        },
        "max_epochs" : 100,
        "total_param_limit" :100_000_000,
        "kernel_regularizer": True,
        "bias_regularizer": True,
        "activity_regularizer": True
    },
    "task2": {
        "_descrip" : "",
        "permitted_layers" : ["InputLayer","Dense","BatchNormalization","Reshape","Flatten", "Dropout", "Conv2D", "MaxPooling2D", "MaxPooling1D", "UpSampling2D"],
        "optimizers" : ["SGD", "Adam", "Nadam","Adadelta","RMSprop"],
        "act_funcs" : ["relu","leaky_relu","sigmoid","elu","linear"],
        "min_lr" :  0.000001,
        "max_lr" :  0.1,
        "layer_limits" : {
            "Dense" : 30
        },
        "layer_mins" : {
        },
        "max_epochs" : 100,
        "total_param_limit" :100_000_000,
        "kernel_regularizer": True,
        "bias_regularizer": True,
        "activity_regularizer": True
    },
    "task3": {
        "_descrip" : "",
        "permitted_layers" : ["InputLayer","Dense","BatchNormalization","Reshape","Flatten", "Dropout", "Conv2D", "MaxPooling2D", "MaxPooling1D", "UpSampling2D"],
        "optimizers" : ["SGD", "Adam", "Nadam","Adadelta","RMSprop"],
        "act_funcs" : ["relu","leaky_relu","sigmoid","elu","linear"],
        "min_lr" :  0.000001,
        "max_lr" :  0.1,
        "layer_limits" : {
            "Dense" : 30
        },
        "layer_mins" : {
        },
        "max_epochs" : 100,
        "total_param_limit" :100_000_000,
        "kernel_regularizer": True,
        "bias_regularizer": True,
        "activity_regularizer": True
    },
}

In [ ]:
import tensorflow as tf
import json
from google.colab import files
import numpy as np
from tensorflow.keras.datasets import mnist
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import wandb
from keras.preprocessing.image import load_img, array_to_img, img_to_array
from sklearn.model_selection import train_test_split
from pathlib import Path
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
def rmse_loss(preds, real):
  return tf.reduce_mean(tf.square(tf.subtract(preds, real)))
class Assignment2:
    def __init__(self):
        self.constraints = cs

    def sample_constraint(self, mod, task, epoch):
        task_constraints = self.constraints[task]
        permitted_layers = task_constraints['permitted_layers']
        layer_limits = task_constraints['layer_limits']
        layer_mins = task_constraints['layer_mins']
        total_param_limit = task_constraints['total_param_limit']
        max_epochs= task_constraints['max_epochs']
        kernel_regularizer = task_constraints['kernel_regularizer']
        bias_regularizer = task_constraints['bias_regularizer']
        activity_regularizer = task_constraints['activity_regularizer']
        regularizers = [ 'kernel_regularizer', 'bias_regularizer','activity_regularizer' ]
        regularizer_allowed = [kernel_regularizer, bias_regularizer, activity_regularizer]
        act_funcs = task_constraints['act_funcs']

        min_lr = task_constraints['min_lr']
        max_lr = task_constraints['max_lr']
        permitted_opts =  task_constraints['optimizers']

        # both trainable and untrainable
        if mod.count_params()>total_param_limit:
            return False, "Too many parameters"
        layer_counts = {}
        config = mod.get_config()
        comp_config = mod.optimizer.get_config()
        if epoch>max_epochs:
            return (False, f"Too many trianing epochs, limit is \'{max_epochs}\'")

        if comp_config['name'] not in permitted_opts:
            return (False, f"Unpermitted Optimizer \'{comp_config['name']}\'")
        if comp_config['learning_rate'] < min_lr and not abs(comp_config['learning_rate'] - min_lr)<1e-5:
            return (False, f"Learning rate {comp_config['learning_rate']} lower than specified minimum \'{min_lr}\'")
        if comp_config['learning_rate'] > max_lr and not abs(comp_config['learning_rate'] - max_lr)<1e-5:
            return (False, f"Learning rate {comp_config['learning_rate']} higher than specified max \'{max_lr}\'")

        for layer in config['layers']:
            l_type = layer['class_name']
            if l_type not in permitted_layers:
                return (False, f"Unpermitted layer type \'{l_type}\'")
            if l_type not in layer_counts: layer_counts[l_type]=0
            layer_counts[l_type]+=1
            if l_type in layer_limits and layer_counts[l_type]>layer_limits[l_type]:
                return (False, f"Limit of \'{l_type}\' layers exceeded")
            if 'activation' in layer['config'] and layer['config']['activation'] not in act_funcs:
              return (False, f"Unpermitted activation function \'{layer['config']['activation']}\'")

            for regularizer, allowed in zip(regularizers,regularizer_allowed):
                if regularizer in layer['config']:
                    if layer['config'][regularizer]!=None and not allowed:
                        return (False, f"Unpermitted regularizer: {regularizer}")
        for l_type in layer_mins:
            if layer_counts[l_type]<layer_mins[l_type]:
                return (False, f"Need to include a minmum of \'{layer_mins[l_type]}\' {l_type} layers")
        return True, "Model satisifes constraints"
    def getTask3SampleImage(self, tpe="train"):
      X = []
      Y = []

      for img in os.listdir("train"):
          img = load_img(f"train/{img}", grayscale=True,target_size=(420,540))
          img = img_to_array(img).astype('float32')/255.
          X.append(img)

      for img in os.listdir("train_cleaned"):
          img = load_img(f"train_cleaned/{img}", grayscale=True,target_size=(420,540))
          img = img_to_array(img).astype('float32')/255.
          Y.append(img)


      X = np.array(X)
      Y = np.array(Y)

      X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1, random_state=111)

      if tpe=='train':
          idx=np.random.choice(list(range(len(X_train))))
          return X_train[idx],y_train[idx]
      else:
        idx=np.random.choice(list(range(len(X_valid))))
        return X_valid[idx],y_valid[idx]

    def trainTaskOne(self, model, run_name,  batch_size):

        (x_train, _), (x_test, _) = mnist.load_data()
        np.random.seed(42)
        np.random.shuffle(x_train)
        split = int(0.75*len(x_train))
        x_valid = x_train[split:,:,:]
        x_train = x_train[:split,:,:]
        x_train = x_train.astype('float32') / 255.
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_valid = x_valid.astype('float32') / 255.
        x_valid = np.reshape(x_valid, (len(x_valid), 28, 28, 1))

        task = "task1"
        project_name = task
        run_name = run_name.strip()
        wandb.init(
        project= project_name,
        name = run_name)
        if not run_name.replace('_','').isalnum():
          raise ValueError("Run name can only contain alphanumeric caracters and _")
        #model.compile(optimizer=optimizer, loss=loss_function)
        callback_ckpt = tf.keras.callbacks.ModelCheckpoint(filepath=task+'/'+task+"_{epoch:03d}",
                                                 save_weights_only=True,
                                                 save_best_only = False,
                                                 monitor="val_loss",
                                                 verbose=1)
        check, msg = self.sample_constraint(model, task, 50)
        if not check:
          raise ValueError(msg)
        filename=f'{task}/history_log.csv'
        if not os.path.exists(task):
            os.makedirs(task)

        callback_hst = AssignmentSubmissionCallback(filename, x_train,x_train)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(f"./tboard/{project_name}/{run_name}", histogram_freq=1)
        model.fit(x_train,
                x_train,
                batch_size=batch_size,
                epochs=50,
                validation_data=(x_valid, x_valid),
                  callbacks=[callback_ckpt,callback_hst,tensorboard_callback,WandbMetricsLogger()], verbose=0)
        wandb.finish()
        model.save(f"./{task}/final_model")
        os.system(f"tar -cf {task}_{run_name}.tar.gz ./{task}")
        files.download(f'{task}_{run_name}.tar.gz')

    def trainTaskTwo(self, model, run_name,  batch_size):
        (x_train, _), (x_test, _) = mnist.load_data()
        np.random.seed(42)
        np.random.shuffle(x_train)
        noise_factor = 0.5
        x_train_noisy = x_train + noise_factor + np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
        x_train_noisy = np.clip(x_train_noisy, 0., 1.)
        split = int(0.75*len(x_train))
        x_valid = x_train[split:,:,:]
        x_train = x_train[:split,:,:]
        x_train = x_train.astype('float32') / 255.
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_valid = x_valid.astype('float32') / 255.
        x_valid = np.reshape(x_valid, (len(x_valid), 28, 28, 1))

        task = "task2"
        project_name = task
        run_name = run_name.strip()
        wandb.init(
        project= project_name,
        name = run_name)
        if not run_name.replace('_','').isalnum():
          raise ValueError("Run name can only contain alphanumeric caracters and _")
        #model.compile(optimizer=optimizer, loss=loss_function)
        callback_ckpt = tf.keras.callbacks.ModelCheckpoint(filepath=task+'/'+task+"_{epoch:03d}",
                                                 save_weights_only=True,
                                                 save_best_only = False,
                                                 monitor="val_loss",
                                                 verbose=1)
        check, msg = self.sample_constraint(model, task, 50)
        if not check:
          raise ValueError(msg)
        filename=f'{task}/history_log.csv'
        if not os.path.exists(task):
            os.makedirs(task)

        callback_hst = AssignmentSubmissionCallback(filename, x_train,x_train)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(f"./tboard/{project_name}/{run_name}", histogram_freq=1)
        model.fit(x_train,
                x_train,
                batch_size=batch_size,
                epochs=50,
                validation_data=(x_valid, x_valid),
                  callbacks=[callback_ckpt,callback_hst,tensorboard_callback,WandbMetricsLogger()], verbose=0)
        wandb.finish()
        model.save(f"./{task}/final_model")
        os.system(f"tar -cf {task}_{run_name}.tar.gz ./{task}")
        files.download(f'{task}_{run_name}.tar.gz')

    def trainTaskThree(self, model, run_name,  batch_size):

        X = []
        Y = []

        for img in os.listdir("train"):
            img = load_img(f"train/{img}", grayscale=True,target_size=(420,540))
            img = img_to_array(img).astype('float32')/255.
            X.append(img)

        for img in os.listdir("train_cleaned"):
            img = load_img(f"train_cleaned/{img}", grayscale=True,target_size=(420,540))
            img = img_to_array(img).astype('float32')/255.
            Y.append(img)


        X = np.array(X)
        Y = np.array(Y)

        X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1, random_state=111)


        task = "task3"
        project_name = task
        run_name = run_name.strip()
        wandb.init(
        project= project_name,
        name = run_name)
        if not run_name.replace('_','').isalnum():
          raise ValueError("Run name can only contain alphanumeric caracters and _")
        #model.compile(optimizer=optimizer, loss=loss_function)
        callback_ckpt = tf.keras.callbacks.ModelCheckpoint(filepath=task+'/'+task+"_{epoch:03d}",
                                                 save_weights_only=True,
                                                 save_best_only = False,
                                                 monitor="val_loss",
                                                 verbose=1)
        check, msg = self.sample_constraint(model, task, 50)
        if not check:
          raise ValueError(msg)
        filename=f'{task}/history_log.csv'
        if not os.path.exists(task):
            os.makedirs(task)

        callback_hst = AssignmentSubmissionCallback(filename, x_train,x_train)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(f"./tboard/{project_name}/{run_name}", histogram_freq=1)
        model.fit(X_train,
                y_train,
                batch_size=batch_size,
                epochs=50,
                validation_data=(X_valid, y_valid),
                  callbacks=[callback_ckpt,callback_hst,tensorboard_callback,WandbMetricsLogger()], verbose=0)
        wandb.finish()
        model.save(f"./{task}/final_model")
        os.system(f"tar -cf {task}_{run_name}.tar.gz ./{task}")
        files.download(f'{task}_{run_name}.tar.gz')

    def setup(self):
      wandb.login()

    def make_sub(self, task, chosen_run1, chosen_run2, just):
      while not os.path.exists(task+'_'+chosen_run1+'.tar.gz'):
          print(f"ERROR: Missing the run! Find the file {task+'_'+chosen_run1}.tar.gz in your downloads, and upload it in the left hand pane and click enter.")
          input()
      while not os.path.exists(task+'_'+chosen_run2+'.tar.gz'):
          print(f"ERROR: Missing the run! Find the file {task+'_'+chosen_run2}.tar.gz in your downloads, and upload it in the left hand pane and click enter.")
          input()

      f=open(f"./explain.txt","w")
      f.write(just)
      f.close()
      os.system(f"tar -cf {task}.tar.gz {task}_{chosen_run1}.tar.gz {task}_{chosen_run2}.tar.gz explain.txt")
      files.download(f'{task}.tar.gz')

    def make_submission_one(self, chosen_run1, chosen_run2, just):
      self.make_sub("task1", chosen_run1, chosen_run2, just)
    def make_submission_two(self, chosen_run1, chosen_run2, just):
      self.make_sub("task2", chosen_run1, chosen_run2, just)
    def make_submission_three(self, chosen_run1, chosen_run2, chosen_run3, just):
      task='task3'
      while not os.path.exists(task+'_'+chosen_run1+'.tar.gz'):
          print(f"ERROR: Missing the run! Find the file {task+'_'+chosen_run1}.tar.gz in your downloads, and upload it in the left hand pane and click enter.")
          input()
      while not os.path.exists(task+'_'+chosen_run2+'.tar.gz'):
          print(f"ERROR: Missing the run! Find the file {task+'_'+chosen_run2}.tar.gz in your downloads, and upload it in the left hand pane and click enter.")
          input()
      while not os.path.exists(task+'_'+chosen_run3+'.tar.gz'):
          print(f"ERROR: Missing the run! Find the file {task+'_'+chosen_run3}.tar.gz in your downloads, and upload it in the left hand pane and click enter.")
          input()

      f=open(f"./explain.txt","w")
      f.write(just)
      f.close()
      os.system(f"tar -cf {task}.tar.gz {task}_{chosen_run1}.tar.gz {task}_{chosen_run2}.tar.gz {task}_{chosen_run3}.tar.gz explain.txt")
      files.download(f'{task}.tar.gz')



class AssignmentSubmissionCallback(tf.keras.callbacks.Callback):
   def __init__(self, filename, x_train, y_train):
        super().__init__()
        self.filename=filename
        self.x_train = x_train
        self.y_train = y_train
        f = open(self.filename, "w")
        f.write("epoch,train_loss,test_loss\n")
        f.close()
   def on_epoch_end(self, epoch, logs=None):
      train_loss = self.model.evaluate(self.x_train, self.y_train, verbose=0)
      if isinstance(train_loss, list): train_loss=train_loss[0]
      test_loss = logs['val_loss']
      f = open(self.filename, "a")
      f.write(f"{epoch},{train_loss},{test_loss}\n")
      f.close()



# First steps

In [ ]:
A = Assignment2()
A.setup()

# CS6421 Assignment 2: Autoencoders

 You will be given a tutorial introduction to the deep autoencoder, and will then need to use this model to solve real-world problems focusing on image denoising.


Please submit your work ideally in a clear Jupyter notebook, highlighting the code that you have written. Present the comparisons and explanations of different model performance results in specific detail.

For each assignment, the following text needs to be agreed to:

 By submitting this exam, I declare

(1) that all work of it is my own;

(2) that I did not seek whole or partial solutions for any part of my submission from others; and

(3) that I did not and will not discuss, exchange, share, or publish complete or partial solutions for this exam or any part of it.

## Introduction: Basic Autoencoder
In this assignment, we will create a **simple autoencoder** model using the [TensorFlow subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing). We start with the popular [MNIST dataset](http://yann.lecun.com/exdb/mnist/) (Grayscale images of hand-written digits from 0 to 9).
_[This first section is based on a notebook orignially contributed by: [afagarap](https://github.com/afagarap)]_

"Autoencoding" is a data compression algorithm where the compression and decompression functions are 1) data-specific, 2) lossy, and 3) learned automatically from examples rather than engineered by a human. Additionally, in almost all contexts where the term "autoencoder" is used, the compression and decompression functions are implemented with neural networks.

 1) Autoencoders are _data-specific_, which means that they will only be able to compress data similar to what they have been trained on. This is different from, say, the MPEG-2 Audio Layer III (MP3) compression algorithm, which only holds assumptions about "sound" in general, but not about specific types of sounds. An autoencoder trained on pictures of faces would do a rather poor job of compressing pictures of trees, because the features it would learn would be face-specific.

2) Autoencoders are _lossy_, which means that the decompressed outputs will be degraded compared to the original inputs (similar to MP3 or JPEG compression). This differs from lossless arithmetic compression.

3) Autoencoders are _learned automatically from data examples_, which is a useful property: it means that it is easy to train specialized instances of the algorithm that will perform well on a specific type of input. It doesn't require any new engineering, just appropriate training data.

To build an autoencoder, you need three things: an encoding function, a decoding function, and a distance function between the amount of information loss between the compressed representation of your data and the decompressed representation (i.e. a "loss" function). The encoder and decoder will be chosen to be parametric functions (typically neural networks), and to be differentiable with respect to the distance function, so the parameters of the encoding/decoding functions can be optimize to minimize the reconstruction loss, using Stochastic Gradient Descent.

In general, a neural network is a computational model that is used for finding a function describing the relationship between data features $x$ and its values or labels $y$, i.e. $y = f(x)$.
An autoencoder is specific type of neural network, which consists of encoder and decoder components: (1) the **encoder**, which learns a compressed data representation $z$, and (2) the **decoder**, which reconstructs the data $\hat{x}$ based on its idea $z$ of how it is structured:
$$ z = f\big(h_{e}(x)\big)$$
$$ \hat{x} = f\big(h_{d}(z)\big),$$
where $z$ is the learned data representation by encoder $h_{e}$, and $\hat{x}$ is the reconstructed data by decoder $h_{d}$ based on $z$.

## Setup
We start by importing the libraries and functions that we will need.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

#try:
  # The %tensorflow_version magic only works in colab.
  # tensorflow_version 2.x
#except Exception:
#  pass
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.datasets import mnist

print('TensorFlow version:', tf.__version__)
print('Is Executing Eagerly?', tf.executing_eagerly())

## Autoencoder model

The encoder and decoder are defined as:
$$ z = f\big(h_{e}(x)\big)$$
$$ \hat{x} = f\big(h_{d}(z)\big),$$
where $z$ is the compressed data representation generated by encoder $h_{e}$, and $\hat{x}$ is the reconstructed data generated by decoder $h_{d}$ based on $z$.

<div align="center"><img src="https://github.com/benjaminirving/mlseminars-autoencoders/blob/master/imgs/d1.png?raw=1" width="80%"></div>

In this figure, we take as input an image, and compress that image before decompressing it using a Dense network. We further define a simple model for this below.

### Define an encoder layer

The first component, the **encoder**, is similar to a conventional feed-forward network. However, it's function is not  predicting values (a _regression_ task) or categories (a _classification_ task). Instead, it's function is to learn a compressed data structure  $z$. We can implement the encoder layer as dense layers, as follows:

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)

    def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)

The _encoding_ is done by passing data input $x$ to the encoder's hidden layer $h$ in order to learn the data representation $z = f(h(x))$.

We first create an `Encoder` class that inherits the [`tf.keras.layers.Layer`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) class to define it as a layer. The compressed layer $z$ is a _component_ of the autoencoder model.

Analyzing the code, the `Encoder` layer is defined to have a single hidden layer of neurons (`self.hidden_layer`) to learn the input features. Then, we connect the hidden layer to a layer (`self.output_layer`) that encodes the learned activations to the lower dimensional layer for $z$.

### Define a decoder layer

The second component, the **decoder**, is also similar to a feed-forward network. However, instead of reducing data to lower dimension, it attempts to reverse the process, i.e. reconstruct the data $\hat{x}$ from its lower dimension representation $z$ to its original dimension.

The _decoding_ is done by passing the lower dimension representation $z$ to the decoder's hidden layer $h$ in order to reconstruct the data to its original dimension $\hat{x} = f(h(z))$. We can implement the decoder layer as follows,

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.relu)

    def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)

We now create a `Decoder` class that also inherits the `tf.keras.layers.Layer`.

The `Decoder` layer is also defined to have a single hidden layer of neurons to reconstruct the input features $\hat{x}$ from the learned representation $z$ by the encoder $f\big(h_{e}(x)\big)$. Then, we connect its hidden layer to a layer that decodes the data representation from lower dimension $z$ to its original dimension $\hat{x}$. Hence, the "output" of the `Decoder` layer is the reconstructed data $\hat{x}$ from the data representation $z$.

Ultimately, the output of the decoder is the autoencoder's output.

Now that we have defined the components of our autoencoder, we can finally build our model.

### Build the autoencoder model

We can now build the autoencoder model by instantiating `Encoder` and `Decoder` layers.

In [ ]:
class Autoencoder(tf.keras.Model):
  def __init__(self, intermediate_dim, original_dim):
    super(Autoencoder, self).__init__()
    self.loss = []
    self.encoder = Encoder(intermediate_dim=intermediate_dim)
    self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)

  def call(self, input_features):
    code = self.encoder(input_features)
    reconstructed = self.decoder(code)
    return reconstructed

As discussed above, the encoder's output is the input to the decoder, as it is written above (`reconstructed = self.decoder(code)`).

## Reconstruction error

To learn the compressed layer $z$, we define a loss function over the difference between the input data $x$ and the reconstruction of $x$, which is $\hat{x}$.
We call this comparison the reconstruction error function, a given by the following equation:
$$ L = \dfrac{1}{n} \sum_{i=0}^{n-1} \big(\hat{x}_{i} - x_{i}\big)^{2}$$
where $\hat{x}$ is the reconstructed data while $x$ is the original data.

In [ ]:
def loss(preds, real):
  return tf.reduce_mean(tf.square(tf.subtract(preds, real)))

## Forward pass and optimization

We will write a function for computing the forward pass, and applying a chosen optimization function.

In [ ]:
def train(loss, model, opt, original):
  with tf.GradientTape() as tape:
    preds = model(original)
    reconstruction_error = loss(preds, original)
  gradients = tape.gradient(reconstruction_error, model.trainable_variables)
  gradient_variables = zip(gradients, model.trainable_variables)
  opt.apply_gradients(gradient_variables)

  return reconstruction_error

## The training loop

Finally, we will write a function to run the training loop. This function will take arguments for the model, the optimization function, the loss, the dataset, and the training epochs.

The training loop itself uses a `GradientTape` context defined in `train` for each batch.

In [ ]:
def train_loop(model, opt, loss, dataset, epochs):
  for epoch in range(epochs):
    epoch_loss = 0
    for step, batch_features in enumerate(dataset):
      loss_values = train(loss, model, opt, batch_features)
      epoch_loss += loss_values
    model.loss.append(epoch_loss)
    print('Epoch {}/{}. Loss: {}'.format(epoch + 1, epochs, epoch_loss.numpy()))

## Process the dataset

Now that we have defined our `Autoencoder` class, the loss function, and the training loop, let's import the dataset. We will normalize the pixel values for each example through dividing by maximum pixel value. We shall flatten the examples from 28 by 28 arrays to 784-dimensional vectors.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train / 255.

x_train = x_train.astype(np.float32)
x_train = np.reshape(x_train, (x_train.shape[0], 784))
x_test = np.reshape(x_test, (x_test.shape[0], 784))

training_dataset = tf.data.Dataset.from_tensor_slices(x_train).batch(256)

## Train the model

Now all we have to do is instantiate the autoencoder model and choose an optimization function, then pass the intermediate dimension and the original dimension of the images.

In [ ]:
model = Autoencoder(intermediate_dim=128, original_dim=784)
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

train_loop(model, opt, loss, training_dataset, 20)

## Plot the in-training performance

Let's take a look at how the model performed during training in a couple of plots.

In [ ]:
plt.plot(range(20), model.loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

## Predictions

Finally, we will look at some of the reconstructions

In [ ]:
number = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for index in range(number):
    # display original
    ax = plt.subplot(2, number, index + 1)
    plt.imshow(x_test[index].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, number, index + 1 + number)
    plt.imshow(model(x_test)[index].numpy().reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 1. Tasks for Basic Autoencoder Assignment

As you may see after training this model, the reconstructed images are quite blurry. A number of things could be done to move forward from this point, e.g. adding more layers, or using a convolutional neural network architecture as the basis of the autoencoder, or use a different kind of autoencoder.

- generate results for a more complex Dense architecturer
- define more complex CNN  architectures (similar to as described) and compare their performance to that of the Dense models

We are also going to move away from using Keras Sequential() models and introduce the [functional API](https://www.tensorflow.org/guide/keras/functional_api). This becomes important later (oustide the scope of this assignment) for implementing techniques such as residual connections.







In [ ]:
def rmse_loss(preds, real):
  return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(preds, real))))

In [ ]:
input_img = tf.keras.layers.Input(shape=(28, 28, 1))

# flatten the input to a vector
x = tf.keras.layers.Flatten()(input_img)

x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dense(32,activation='relu')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dense(256,activation='relu')(x)

x = tf.keras.layers.Dense(784,activation='relu')(x)
# turn the final layer back into an image
x = tf.keras.layers.Reshape((28,28,1))(x)

autoencoder = tf.keras.models.Model(input_img, x)
autoencoder.compile(optimizer='adadelta', loss=rmse_loss)
autoencoder.summary()

In [ ]:
run_name = "basic_dense_xxxxx"
A.trainTaskOne(autoencoder,run_name,32)

Since our inputs are images, it makes sense to try to use convolutional neural networks (CNNs) as encoders and decoders. In practical settings, autoencoders applied to images are always convolutional autoencoders -- they simply perform much better.

- implement a CNN model, where the encoder will consist of a stack of Conv2D and MaxPooling2D layers (max pooling being used for spatial down-sampling), while the decoder will consist in a stack of Conv2D and UpSampling2D layers. To improve the quality of the reconstructed image, we use more filters per layer. Example model details are:

In [ ]:
input_img = tf.keras.layers.Input(shape=(28, 28, 1)) # adapt this if using `channels_first` image data format

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.models.Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss=rmse_loss)

In [ ]:
run_name = "basic_cnn_6_layers"
A.trainTaskOne(autoencoder,run_name,32)


- train the model for 50 epochs and compare the results to the model where you use a dense encoding rather than convolutions.

**Scoring**: 10 marks total
- models [5 marks]:
  - Dense, multi-layer model
  - CNN complex model
- results and discussion [5 marks]: present the results in a clear fashion and explain why the results were as obtained. Good experimental design and statistical significance testing will be rewarded.

Submit your best Dense and CNN based models, as well as well as the explanation of results.

In [ ]:
''' SUBMISSION '''
bestDenseModel = 'basic_dense_xxxxx'
bestCNNModel = 'basic_cnn_6_layers'
explanation = '''
    The CNN model  performed significantly better across runs, etc
'''

A.make_submission_one(bestDenseModel, bestCNNModel, explanation)

## 2. Denoising autoencoder

For this real-world application, we will use an autoencoder to remove noise from an image. To do this, we

- learn a more robust representation by forcing the autoencoder to learn an input from a corrupted version of itself

<div align="center"><img src="https://github.com/benjaminirving/mlseminars-autoencoders/blob/master/imgs/d3.png?raw=1" width="60%"></div>

The first step: generate synthetic noisy digits as follows: apply a gaussian noise matrix and clip the images between 0 and 1.

This code will be handled within the Assignment class - but is also illustrated below for your own learning



In [ ]:

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Introduce noise with a probability factor of 0.5
noise_factor = 0.5
x_train_noisy = x_train + noise_factor + np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor + np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Next, plot some figures to see what the digits look like with noise added.

In [ ]:
# Plot figures to show what the noisy digits look like
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Models
- Train CNN and Dense model for 50 epochs and compare the results to the model where you use a dense encoding rather than convolutions. Identify the best configurations for each, and explain their differences in performance.

In [ ]:
def rmse_loss(preds, real):
  return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(preds, real))))

In [ ]:
input_img = tf.keras.layers.Input(shape=(28, 28, 1))

# flatten the input to a vector
x = tf.keras.layers.Flatten()(input_img)

x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dense(32,activation='relu')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dense(256,activation='relu')(x)

x = tf.keras.layers.Dense(784,activation='relu')(x)
# turn the final layer back into an image
x = tf.keras.layers.Reshape((28,28,1))(x)

autoencoder = tf.keras.models.Model(input_img, x)
autoencoder.compile(optimizer='adadelta', loss=rmse_loss)
autoencoder.summary()

In [ ]:
# This will train for 50 epochs
run_name = "basic_dense_6_layers"
A.trainTaskTwo(autoencoder,run_name,128)

In [ ]:
input_img = tf.keras.layers.Input(shape=(28, 28, 1)) # adapt this if using `channels_first` image data format

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.models.Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss=rmse_loss)

In [ ]:
# This will train for 50 epochs
run_name = "basic_cnn_6_layers"
A.trainTaskTwo(autoencoder,run_name,128)

**Scoring**: 10 marks total
- models [5 marks]:
  - Dense, multi-layer model
  - CNN complex model
- results and discussion [5 marks]: present the results in a clear fashion and explain why the results were as obtained. Good experimental design and statistical significance testing will be rewarded.

In [ ]:
''' SUBMISSION '''
bestDenseModel = 'basic_dense_6_layers'
bestCNNModel = 'basic_cnn_6_layers'
explanation = '''
    The CNN model  performed significantly better across runs, etc
    ... deep autoencoders performed worse/better than shallow, ....
'''

A.make_submission_two(bestDenseModel, bestCNNModel, explanation)

**3. Text Reconstruction Application**

You will now use the approach just described to reconstruct corrupted text. We will use a small dataset with grey-scale images of size $420 \times 540$.  The steps required are as follows:

- Apply this autoencoder approach (as just described) to the text data provided as noted below.
- You must create an algorithm to clean the images in the test set, and report the error as RMSE (root-mean-square error).

**Scoring**: 40 marks total
- models [25 marks]:
  - Dense, multi-layer model [5 marks];
  - CNN basic model [5 marks];
  - CNN complex model [15 marks].
- results and discussion [15 marks]: present the results in a clear fashion and explain why the results were as obtained. Good experimental design and statistical significance testing will be rewarded.


You will get full marks if you achieve RMSE < 0.005. Deductions are as follows:
- -1: 0.01 $\leq$ RMSE $\leq$ 0.005
- -5: 0.05 $\leq$ RMSE $\leq$ 0.01
- -10:  RMSE $>$ 0.05

Deductions are made if **none** of your models attain the speecifid RMSE losses above



Next you must build the model. I provide the code framework, with the model details left up to you.

In [ ]:
input_img = tf.keras.layers.Input(shape=(420,540,1))
#enoder
# enter encoder model here

#decoder
# enter decoder model model


# Compile
autoencoder.compile(optimizer='adam', loss=rmse_loss)

In [ ]:
input_img = tf.keras.layers.Input(shape=(420,540,1)) # adapt this if using `channels_first` image data format

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.models.Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss=rmse_loss)

Next we have code to compile and run the model. Please modify the code below to fit your purposes.

In [ ]:
# This will train for 50 epochs
run_name = "basic_cnn_x_layers"
A.trainTaskThree(autoencoder,run_name,128)

Next we can look at some predictions from the trained model.

In [ ]:
# Compute the prediction
noisy_sample, clean_sample = A.getTask3SampleImage("train")
predicted_label = np.squeeze(autoencoder.predict(noisy_sample.reshape(1,420,540,1)))

f, ax = plt.subplots(1,2, figsize=(10,8))
ax[0].imshow(np.squeeze(clean_sample.reshape(1,420,540,1)), cmap='gray')
ax[1].imshow(np.squeeze(predicted_label.astype('int8')), cmap='gray')
plt.show()


In [ ]:
# Compute the prediction
noisy_sample, clean_sample = A.getTask3SampleImage("valid")
predicted_label = np.squeeze(autoencoder.predict(noisy_sample.reshape(1,420,540,1)))

f, ax = plt.subplots(1,2, figsize=(10,8))
ax[0].imshow(np.squeeze(clean_sample.reshape(1,420,540,1)), cmap='gray')
ax[1].imshow(np.squeeze(predicted_label.astype('int8')), cmap='gray')
plt.show()

To complete this part of the project, you need to examine methods to improve the quality of the predictions. Report on methods to get better performance on this task.

In [ ]:
''' SUBMISSION '''
bestDenseModel = 'basic_dense_6_layers'
bestSimpleCNNModel = 'basic_cnn_x_layers'
bestComplexCNNModel = 'basic_cnn_x_layers'
explanation = '''
    The CNN model  performed significantly better across runs, etc
    ... deep autoencoders performed worse/better than shallow, ....
'''

A.make_submission_three(bestDenseModel, bestSimpleCNNModel, bestComplexCNNModel, explanation)

# Assignment Submission
1. Save this notebook (Ctrl+S) and download it with File->Download->Download .ipynb
2. Submit both this .ipynb and **task1.tar.gz, task2.tar.gz and task3.tar.gz** to Canvas - do not zip, tar or otherwsie combine or modify these files

## References
* Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo, Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis, Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow, Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia, Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster, Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens, Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker, Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas, Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke, Yuan Yu, and Xiaoqiang Zheng. TensorFlow: Large-scale machine learning on heterogeneous systems, 2015. Software available from [tensorflow.org](https://tensorflow.org/).
* Chollet, F. (2016, May 14). Building Autoencoders in Keras. Retrieved March 19, 2019, from https://blog.keras.io/building-autoencoders-in-keras.html
* Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep learning. MIT press.